## Filtering data from API Requests

In [401]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient 
import geopandas as gdp
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from functools import reduce
import operator

In [402]:
client = MongoClient()
db = client.crunchbase
companies = db.companies
offices = db.offices
starbucks = db.starbucks
starbucks1 = db.starbucks1

In [403]:
load_dotenv()

True

In [404]:
token1=os.getenv("tok1")
print(token1)
token2=os.getenv("tok2")
print(token2)

0UGDAD3T53PSEKRJRXPVE52FWO0T35OO3SNGTV41HLOL42BV
CWRVBC1CFSU4KS5C4CINS44MYLUYK244LY4T1U1R51KDJ504


### 1. Api Requests

In [405]:
url = "https://favqs.com/api/session"

In [406]:
netbiscuits = {'type': 'Point', 'coordinates': [-122.4264417, 37.7550372] }

In [407]:
ecolect = {'type': 'Point', 'coordinates': [-122.410111, 37.776769] }

In [408]:
popego = {'type': 'Point', 'coordinates': [-122.421242, 37.778687]}

In [435]:
digg = {'type': 'Point', 'coordinates': [-122.394523, 37.764726] }

In [410]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
karaoke = "5744ccdfe4b0c0459246b4bb"
music_venue = "4bf58dd8d48988d1e5931735"

In [645]:
def CallApi(point):
    parametros = {"client_id" : f"{token1}",
              "client_secret" : f"{token2}",
              "v": "20180323",
              "ll": f"{point.get('coordinates')[1]},{point.get('coordinates')[0]}",
              "query":"rock climbing spot",
              "limit": 100 
}
    resp = requests.get(url= url_query, params= parametros)
    data = json.loads(resp.text)
    return data

In [465]:
net_nur = CallApi(netbiscuits)
net_nur= net_nur.get("response")
net_nur = net_nur.get("groups")[0]
net_nur= net_nur.get("items")

In [432]:
eco_nur = CallApi(ecolect)
eco_nur = eco_nur.get("response")
eco_nur = eco_nur.get("groups")[0]
eco_nur = eco_nur.get("items")

In [449]:
po_call = CallApi(popego)
detracting = po_call.get("response")
detract = detracting.get("groups")[0]
po_nurs = detract.get("items")

In [436]:
digg_nur = CallApi(digg)
digg_nur= digg_nur.get("response")
digg_nur = digg_nur.get("groups")[0]
digg_nur = digg_nur.get("items")

***Accesing nested dictionnaries:*** 

In [451]:
venue_name = ["venue", "name"]
v_lat = ["venue", "location", "lat"]
v_lng = ["venue", "location", "lng"]

In [456]:
def getValues (dic, maps):
    return reduce (operator.getitem,maps,dic)

In [666]:
    eco_rock_spotting = []
    for dic in eco_rock: 
        venue_name = ["venue", "name"]
        v_lat = ["venue", "location", "lat"]
        v_lng = ["venue", "location", "lng"]
        v_dist = ["venue", "location", "distance"]
        dicc = {}
        dicc["name"] = getValues(dic, venue_name)
        dicc["latitude"]= getValues(dic, v_lat)
        dicc["longitude"] = getValues(dic, v_lng)
        dicc["distance"]= getValues(dic, v_dist)
        eco_rock_spotting.append(dicc)

### 1.1 Looking for nurseries:

In [819]:
dfpo_nurs = pd.DataFrame(po_nursery_spotting)
dfpo_nurs.head()

,name,latitude,longitude,distance
0,C5 Children's School,37.778864,-122.414584,586
1,Judith Baker CD,37.776839,-122.411802,855
2,Kids by the Bay - Civic Center,37.779430,-122.411819,833
3,Japanese YWCA Building / Nihonmachi Little Fri...,37.787017,-122.430609,1240
4,Morning Glory Nursery School,37.776355,-122.443699,1992


In [493]:
dfeco_nurs.insert(4, "company", "Ecolect")

In [496]:
ttl_nursery = [dfnet_nurs, dfpo_nurs, dfdigg_nurs, dfeco_nurs]

In [497]:
ofi_nursery = pd.concat(ttl_nursery)

In [498]:
ofi_nursery.groupby("company")["distance"].count()

company
Digg            5
Ecolect        16
Netbiscuits    19
Popego         13
Name: distance, dtype: int64

In [499]:
ofi_nursery.groupby("company")["distance"].min()

company
Digg            828
Ecolect         149
Netbiscuits    1267
Popego          586
Name: distance, dtype: int64

In [500]:
filt_nursery = ofi_nursery[ofi_nursery['distance'] < 5000]

In [620]:
count_nursery = pd.DataFrame(filt_nursery.groupby("company")["category"].count().sort_values())

In [622]:
count_nursery.columns = ["Nurseries < 5000m"]

In [501]:
filt_nursery["category"] = "nursery"

<ipython-input-501-903399b2a1e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_nursery["category"] = "nursery"


In [558]:
filt_nursery.head()

,name,latitude,longitude,distance,company,category
0,Neighborhood Playgarden,37.756829,-122.440666,1267,Netbiscuits,nursery
1,Rocky Mountain School,37.766022,-122.438141,1598,Netbiscuits,nursery
2,Judith Baker CD,37.776839,-122.411802,2747,Netbiscuits,nursery
3,Morning Glory Nursery School,37.776355,-122.443699,2817,Netbiscuits,nursery
4,C5 Children's School,37.778864,-122.414584,2850,Netbiscuits,nursery


In [560]:
filt_nursery.groupby("company")["category"].count().sort_values()

company
Digg            5
Popego         13
Ecolect        16
Netbiscuits    17
Name: category, dtype: int64

### 1.2 Now let´s look for  starbucks in the area:

In [506]:
net_sbucks = CallApi(netbiscuits)
net_sbucks= net_sbucks.get("response")
net_sbucks = net_sbucks.get("groups")[0]
net_sbucks= net_sbucks.get("items")

In [507]:
eco_sbucks = CallApi(ecolect)
eco_sbucks = eco_sbucks.get("response")
eco_sbucks = eco_sbucks.get("groups")[0]
eco_sbucks = eco_sbucks.get("items")

In [508]:
po_sbucks = CallApi(popego)
po_sbucks = po_sbucks.get("response")
po_sbucks = po_sbucks.get("groups")[0]
po_sbucks = po_sbucks.get("items")

In [509]:
digg_sbucks = CallApi(digg)
digg_sbucks= digg_sbucks.get("response")
digg_sbucks = digg_sbucks.get("groups")[0]
digg_sbucks = digg_sbucks.get("items")

In [544]:
dfdigg_sbucks = pd.DataFrame(digg_sbucks_spotting)

In [545]:
dfdigg_sbucks["company"] = "Popego"

In [546]:
dfdigg_sbucks["category"] = "starbucks"

In [550]:
dfdigg_sbucks.head()

,name,latitude,longitude,distance,company,category
0,Starbucks,37.767287,-122.393926,289,Popego,starbucks
1,Starbucks,37.771196,-122.389364,851,Popego,starbucks
2,Starbucks,37.765878,-122.403449,795,Popego,starbucks
3,Starbucks,37.778704,-122.396727,1568,Popego,starbucks
4,Starbucks,37.766857,-122.409428,1332,Popego,starbucks


In [780]:
dfdigg_sbucks["company"]= "Digg"

In [783]:
ttal_sbucks = [dfnet_sbucks, dfeco_sbucks, dfpo_sbucks, dfdigg_sbucks]

In [784]:
ofi_sbucks = pd.concat(ttal_sbucks)

In [785]:
sbucks_filtered = ofi_sbucks[ofi_sbucks["distance"] < 1000]

In [786]:
count_sbucks = pd.DataFrame(sbucks_filtered.groupby("company")["category"].count().sort_values())

In [787]:

count_sbucks.columns = ["Starbucks < 1000m"]

#### Joining DF:

In [815]:
total_locations = [sbucks_filtered,filt_nursery]

In [790]:
data_map = pd.concat(total_locations)

In [791]:
data_map

,name,latitude,longitude,distance,company,category
0,Starbucks,37.760941,-122.434608,973,Netbiscuits,starbucks
1,Starbucks,37.751373,-122.431825,625,Netbiscuits,starbucks
0,Starbucks,37.784143,-122.407596,850,Ecolect,starbucks
2,Starbucks,37.777090,-122.417298,633,Ecolect,starbucks
6,Starbucks,37.770190,-122.403465,937,Ecolect,starbucks
...,...,...,...,...,...,...
11,Friends Of Potrero Hill Nursery School,37.758353,-122.389650,828,Ecolect,nursery
12,Judith Baker CD,37.776839,-122.411802,2032,Ecolect,nursery
13,C5 Children's School,37.778864,-122.414584,2364,Ecolect,nursery
14,Kids by the Bay - Civic Center,37.779430,-122.411819,2235,Ecolect,nursery


### 1.3 Karaoke

In [572]:
net_karaoke = CallApi(netbiscuits)
net_karaoke= net_karaoke.get("response")
net_karaoke = net_karaoke.get("groups")[0]
net_karaoke= net_karaoke.get("items")

In [573]:
eco_karaoke = CallApi(ecolect)
eco_karaoke = eco_karaoke.get("response")
eco_karaoke = eco_karaoke.get("groups")[0]
eco_karaoke = eco_karaoke.get("items")

In [574]:
po_karaoke = CallApi(popego)
po_karaoke = po_karaoke.get("response")
po_karaoke = po_karaoke.get("groups")[0]
po_karaoke = po_karaoke.get("items")

In [575]:
digg_karaoke = CallApi(digg)
digg_karaoke= digg_karaoke.get("response")
digg_karaoke = digg_karaoke.get("groups")[0]
digg_karaoke = digg_karaoke.get("items")

In [590]:
dfnet_kara = pd.DataFrame(net_kara_spotting)

In [592]:
dfnet_kara.head()

,name,latitude,longitude,distance
0,500 Club,37.763127,-122.424067,924
1,El Rio,37.746845,-122.419465,1099
2,Martuni's,37.772180,-122.422838,1934
3,Amnesia,37.759309,-122.421232,660
4,Dalva,37.764749,-122.422434,1137


In [600]:
dfeco_kara["company"] = "Ecolect"

In [601]:
dfeco_kara["category"] = "karaoke"

In [602]:
ttal_kara = [dfdigg_kara, dfeco_kara, dfnet_kara, dfpo_kara]

In [603]:
karaokes = pd.concat(ttal_kara)

In [604]:
karaokes.head()

,name,latitude,longitude,distance,company,category
0,Cat Club,37.775324,-122.409901,1795,Digg,karaoke
1,Martuni's,37.772180,-122.422838,2626,Digg,karaoke
2,500 Club,37.763127,-122.424067,2606,Digg,karaoke
3,RVIP Lounge & Karoke Cabaret,37.775855,-122.408851,1767,Digg,karaoke
4,Butter,37.771127,-122.413315,1800,Digg,karaoke


In [606]:
karaoke_filt= karaokes[karaokes["distance"]<3000]

In [631]:
count_kara = pd.DataFrame(karaoke_filt.groupby("company")["category"].count().sort_values())

In [633]:
count_kara.columns = ["Karaoke < 3000m"]

### 4. Rock climbiing spots

In [816]:
net_rock = CallApi(netbiscuits)
net_rock= net_rock.get("response")
net_rock = net_rock.get("groups")[0]
net_rock= net_rock.get("items")

In [648]:
eco_rock = CallApi(ecolect)
eco_rock = eco_rock.get("response")
eco_rock = eco_rock.get("groups")[0]
eco_rock = eco_rock.get("items")

In [649]:
po_rock = CallApi(popego)
po_rock = po_rock.get("response")
po_rock = po_rock.get("groups")[0]
po_rock = po_rock.get("items")

In [650]:
digg_rock = CallApi(digg)
digg_rock= digg_rock.get("response")
digg_rock = digg_rock.get("groups")[0]
digg_rock = digg_rock.get("items")

In [667]:
dfeco_rock = pd.DataFrame(eco_rock_spotting)

In [687]:
dfeco_rock["company"] = "Ecolect"

In [682]:
dfdigg_rock["category"] = "sport"

In [683]:
dfdigg_rock

,name,latitude,longitude,distance,company,category
0,The Tree Fort,37.763276,-122.444926,4438,Digg,sport
1,Recology Mixed Inenerts Pile,37.708714,-122.397301,6240,Digg,sport


In [688]:
ttal= [dfnet_rock, dfpo_rock, dfeco_rock, dfdigg_rock]

In [689]:
rocks = pd.concat(ttal)

In [691]:
rocks_filt = rocks[rocks["distance"]<5000]

### 4.Counting venues/office:


In [795]:
counting = [count_sbucks, count_nursery, count_kara]

In [797]:
data_count = pd.concat(counting)

In [818]:
data_count.head()

,Starbucks < 1000m,Nurseries < 5000m,Karaoke < 3000m,Rockclimbing spots < 5000m
company,,,,
Digg,4.0,5.0,26.0,1.0
Ecolect,5.0,16.0,17.0,1.0
Netbiscuits,2.0,17.0,24.0,1.0
Popego,2.0,13.0,31.0,1.0


In [770]:
count_rocks = pd.DataFrame(rocks_filt.groupby("company")["category"].count().sort_values())

In [773]:
count_rocks

,Rockclimbing spots < 5000m
company,
Digg,1
Ecolect,1
Netbiscuits,1
Popego,1


In [772]:
count_rocks.columns = ["Rockclimbing spots < 5000m"]

In [799]:
total = [data_count, count_rocks]

In [800]:
total_count = pd.concat(total)

In [817]:
total_count.head()

,Starbucks < 1000m,Nurseries < 5000m,Karaoke < 3000m,Rockclimbing spots < 5000m
company,,,,
Netbiscuits,2.0,NaN,NaN,NaN
Popego,2.0,NaN,NaN,NaN
Digg,4.0,NaN,NaN,NaN
Ecolect,5.0,NaN,NaN,NaN
Digg,NaN,5.0,NaN,NaN


In [803]:
data_count = pd.DataFrame(total_count.groupby("company")[["Starbucks < 1000m", "Nurseries < 5000m", "Karaoke < 3000m", "Rockclimbing spots < 5000m"]].max())

In [806]:
data_count

,Starbucks < 1000m,Nurseries < 5000m,Karaoke < 3000m,Rockclimbing spots < 5000m
company,,,,
Digg,4.0,5.0,26.0,1.0
Ecolect,5.0,16.0,17.0,1.0
Netbiscuits,2.0,17.0,24.0,1.0
Popego,2.0,13.0,31.0,1.0


In [804]:
data_count.to_csv("data_count.csv")

***Karaoke***: 0.35

***Nurseries***: will commpute for 0.30

***Rockclimbing***: 0.20

***Starbucks***: 0.15

In [1]:
pd.DataFrame(data_count.groupby("company")[["Starbucks < 1000m","Nurseries < 5000m", "Karaoke < 3000m", "Rockclimbing spots < 5000m"]].agg([("Starbucks < 1000m", lambda x: x*0.15), ("Nurseries < 5000m", lambda x: x*0.20), ("Karaoke < 3000m", lambda x: x*0.35), ("Rockclimbing spots < 5000m", lambda x: x*0.20)]) 
             

SyntaxError: unexpected EOF while parsing (<ipython-input-1-ebda580415a7>, line 2)

In [808]:
data_count["Starbucks < 1000m"].apply(lambda x: x*0.15 )

company
Digg           0.60
Ecolect        0.75
Netbiscuits    0.30
Popego         0.30
Name: Starbucks < 1000m, dtype: float64

## 2. Joining Mongo + API data:

### 2.1 Joining all API´s DF:

In [805]:
datas = [data_map,karaoke_filt, rocks_filt]

In [693]:
data_mapa = pd.concat(datas)

In [698]:
data_mapa

,name,latitude,longitude,distance,company,category
0,Starbucks,37.760941,-122.434608,973,Netbiscuits,starbucks
1,Starbucks,37.751373,-122.431825,625,Netbiscuits,starbucks
0,Starbucks,37.784143,-122.407596,850,Ecolect,starbucks
2,Starbucks,37.777090,-122.417298,633,Ecolect,starbucks
6,Starbucks,37.770190,-122.403465,937,Ecolect,starbucks
...,...,...,...,...,...,...
30,Milo Lounge,37.785645,-122.430062,1096,Popego,karaoke
0,The Tree Fort,37.763276,-122.444926,1867,Netbiscuits,sport
0,The Tree Fort,37.763276,-122.444926,2699,Popego,sport
0,The Tree Fort,37.763276,-122.444926,3411,Ecolect,sport


In [615]:
data_mapa.groupby("category")["company"].count()

category
karaoke      98
nursery      51
starbucks    13
Name: company, dtype: int64

In [699]:
data_mapa.to_csv("api_requests.csv")

### 2.2 Adding Mongo:

In [738]:
mongo = pd.read_csv("mongo_search.csv")

In [740]:
del mongo["Unnamed: 0"]

In [742]:
mongo["category"] = "offices"

In [744]:
mongo_api = [data_mapa, mongo]

In [745]:
data = pd.concat(mongo_api)

In [747]:
data.to_csv("all_data.csv")